# Exercise Set 12: Linear regression models.

*Afternoon, August 19, 2019*

In this Exercise Set 12 we will work with linear regression models.

We import our standard stuff. Notice that we are not interested in seeing the convergence warning in scikit-learn so we suppress them for now.

In [39]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

%matplotlib inline

## Exercise Section 12.1: Estimating linear models with gradient decent
 
Normally we use OLS to estimate linear models. In this exercise we replace the OLS-estimator with a new estimator that we code up from scratch. We solve the numerical optimization using the gradient decent algorithm. Using our algorithm we will fit it to some data, and compare our own solution to the standard solution from `sklearn`

> **Ex. 12.1.0**: Import the dataset `tips` from the `seaborn`.


*Hint*: use the `load_dataset` method in seaborn

In [2]:
# [Answer to Ex. 12.1.0]
tips = sns.load_dataset('tips')

> **Ex. 12.1.1**: Convert non-numeric variables to dummy variables for each category (remember to leave one column out for each catagorical variable, so you have a reference). Restructure the data so we get a dataset `y` containing the variable tip, and a dataset `X` containing the 
features. 

>> *Hint*: You might want to use the `get_dummies` method in pandas, with the `drop_first = True` parameter. 

In [3]:
# [Answer to Ex. 12.1.1]
tips_cat = pd.get_dummies(tips, drop_first=True)
tips_cat
y = tips_cat['tip']
y
X = tips_cat.iloc[:,2:]
X

,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
0,2,1,1,0,0,1,1
1,3,0,1,0,0,1,1
2,3,0,1,0,0,1,1
3,2,0,1,0,0,1,1
4,4,1,1,0,0,1,1
5,4,0,1,0,0,1,1
6,2,0,1,0,0,1,1
7,4,0,1,0,0,1,1
8,2,0,1,0,0,1,1
9,2,0,1,0,0,1,1


> **Ex. 12.1.2**: Divide the features and target into test and train data. Make the split 50 pct. of each. The split data should be called `X_train`, `X_test`, `y_train`, `y_test`.

>> *Hint*: You may use `train_test_split` in `sklearn.model_selection`.

In [4]:
# [Answer to Ex. 12.1.2]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)

> **Ex. 12.1.3**: Normalize your features by converting to zero mean and one std. deviation.

>> *Hint 1*: Take a look at `StandardScaler` in `sklearn.preprocessing`. 

>> *Hint 2*: If in doubt about which distribution to scale, you may read [this post](https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i).

In [5]:
# [Answer to Ex. 12.1.3]
print(X_train.head())
scaler =StandardScaler(copy=True, with_mean=False, with_std=True)

scaler.fit(X_train)
print(X_train.head())

     size  sex_Female  smoker_No  day_Fri  day_Sat  day_Sun  time_Dinner
159     4           0          1        0        0        1            1
2       3           0          1        0        0        1            1
59      4           0          1        0        1        0            1
149     2           0          1        0        0        0            0
227     4           0          1        0        1        0            1
     size  sex_Female  smoker_No  day_Fri  day_Sat  day_Sun  time_Dinner
159     4           0          1        0        0        1            1
2       3           0          1        0        0        1            1
59      4           0          1        0        1        0            1
149     2           0          1        0        0        0            0
227     4           0          1        0        1        0            1


/Users/michaelemcken/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


> **Ex. 12.1.4**: Make a function called `compute_error` to compute the prediction errors given input target `y_`, input features `X_` and input weights `w_`. You should use matrix multiplication.
>
>> *Hint 1:* You can use the net-input fct. from yesterday.
>>
>> *Hint 2:* If you run the following code,
>> ```python
y__ = np.array([1,1])
X__ = np.array([[1,0],[0,1]])
w__ = np.array([0,1,1])
compute_error(y__, X__, w__)
```

>> then you should get output:
```python 
array([0,0])
```



In [6]:
# [Answer to Ex. 12.1.4]
y_ = np.array([1,1])
X_ = np.array([[1,0],[0,1]])
w_ = np.array([0,1,1])

def compute_error(X, y, W):
    # compute net-input
    z = W[0] + X.dot(W[1:])

    # unit step-function
    predict = z > 0 # compute prediction (boolean)
    y_hat = np.where(predict, 1, -1)  # convert prediction

    # compute errors
    e = y - y_hat
    SSE = e.T.dot(e)

    return e

compute_error(X_, y_, w_)

array([0, 0])

> **Ex. 12.1.5**: Make a function to update the weights given input target `y_`, input features `X_` and input weights `w_` as well as learning rate, $\eta$, i.e. greek `eta`. You should use matrix multiplication.

In [7]:
# [Answer to Ex. 12.1.5]
def update_weights(X, Y , W, eta):
    def predict(X, Y, W):
        z = W[0] + X.dot(W[1:])
        result = z >= 0
        return result
        
    
    for x, y in zip(X, Y):
            update = eta * (y - predict(x, Y, W))
            W[1:] = W[1:] + update * x
            W[0] = W[0] + update
            
    return W

In [8]:
weights = update_weights(X_,y_,w_,0.1)
print(weights)

[0 1 1]


> **Ex. 12.1.6**: Use the code below to initialize weights `w` at zero given feature set `X`. Notice how we include an extra weight that includes the bias term. Set the learning rate `eta` to 0.001. Make a loop with 50 iterations where you iteratively apply your weight updating function. 

>```python
w = np.zeros(1+X.shape[1])
```

In [9]:
# [Answer to Ex. 12.1.6]
w = np.zeros(1+X_.shape[1])

weights = update_weights(X_,y_,w,0.001)
print(weights)

[0. 0. 0.]


> **Ex. 12.1.7**: Make a function to compute the mean squared error. Alter the loop so it makes 100 iterations and computes the MSE for test and train after each iteration, plot these in one figure. 

>> Hint: You can use the following code to check that your model works:
>>```python
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
assert((w[1:] - reg.coef_).sum() < 0.01)
```

In [10]:
# [Answer to Ex. 12.1.7]

The following bonus exercises are for those who have completed all other exercises until now and have a deep motivation for learning more.

> **Ex. 12.1.8 (BONUS)**: Implement your linear regression model as a class.

> **Ex. 12.1.9 (BONUS)**: Is it possible to adjust our linear model to become a Lasso? Is there a simple fix?

## Exercise Section 12.2: Houseprices
In this example we will try to predict houseprices using a lot of variable (or features as they are called in Machine Learning). We are going to work with Kaggle's dataset on house prices, see information [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). Kaggle is an organization that hosts competitions in building predictive models.

> **Ex. 12.2.0:** Load the california housing data with scikit-learn using the code below. Inspect the data set. 

In [20]:
# [Answer to Ex. 12.2.0]
cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)

X_train.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
10089,4.0893,35.0,5.267760,0.983607,1056.0,2.885246
2136,3.7578,24.0,5.061538,0.957692,781.0,3.003846
17546,2.4306,39.0,4.899209,1.069170,1990.0,3.932806
10051,3.2813,10.0,6.030928,1.159794,537.0,2.768041
3627,4.0950,36.0,5.407166,0.980456,1225.0,3.990228



> **Ex.12.2.1**: Generate interactions between all features to third degree, make sure you **exclude** the bias/intercept term. How many variables are there? Will OLS fail? 

> After making interactions rescale the features to have zero mean, unit std. deviation. Should you use the distribution of the training data to rescale the test data?  

>> *Hint 1*: Try importing `PolynomialFeatures` from `sklearn.preprocessing`

>> *Hint 2*: If in doubt about which distribution to scale, you may read [this post](https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i).

In [38]:
# [Answer to Ex. 12.2.1]

n_degrees = 3
degrees = range(n_degrees+1)

for p in degrees:
    X_train_p = PolynomialFeatures(degree=p, include_bias = False).fit_transform(X_train)
    X_test_p = PolynomialFeatures(degree=p, include_bias = False).fit_transform(X_test)

    
print('We have 6 features/variables in both the training and test set')
print('OLS will fail due to perfect multicollinarity. This leads to the computer not being able to invert the matrixes correctly') 

print(X_train.head())
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(X_train)

We have 6 features/variables in both the training and test set
OLS will fail due to perfect multicollinarity. This leads to the computer not being able to invert the matrixes correctly
       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup
10089  4.0893      35.0  5.267760   0.983607      1056.0  2.885246
2136   3.7578      24.0  5.061538   0.957692       781.0  3.003846
17546  2.4306      39.0  4.899209   1.069170      1990.0  3.932806
10051  3.2813      10.0  6.030928   1.159794       537.0  2.768041
3627   4.0950      36.0  5.407166   0.980456      1225.0  3.990228


StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
# [Answer to Ex. 12.2.2]

lr = LinearRegression()
lr.fit(X_train, y_train)
rr = Ridge(alpha=0.01) 
# higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles

rr.fit(X_train, y_train) 
rr100 = Ridge(alpha=100) #  comparison with alpha value
rr100.fit(X_train, y_train)
train_score = lr.score(X_train, y_train)

test_score=lr.score(X_test, y_test)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
Ridge_train_score100 = rr100.score(X_train,y_train)
Ridge_test_score100 = rr100.score(X_test, y_test)
print ("linear regression train score:", train_score)

print ("linear regression test score:", test_score)
print ("ridge regression train score low alpha:", Ridge_train_score)
print ("ridge regression test score low alpha:", Ridge_test_score)
print ("ridge regression train score high alpha:", Ridge_train_score100)
print ("ridge regression test score high alpha:", Ridge_test_score100)


plt.plot(rr.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Ridge; $\alpha = 0.01$',zorder=7) # zorder for ordering the markersplt.plot(rr100.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Ridge; $\alpha = 100$') # alpha here is for transparencyplt.plot(lr.coef_,alpha=0.4,linestyle='none',marker='o',markersize=7,color='green',label='Linear Regression')plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.show()

> **Ex.12.2.2**: Estimate the Lasso model on the train data set, using values of $\lambda$ in the range from $10^{-4}$ to $10^4$. For each $\lambda$  calculate and save the Root Mean Squared Error (RMSE) for the test and train data. 

> *Hint*: use `logspace` in numpy to create the range.


In [42]:
print(X_train.head(5))
print(y_train)

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup
10089  4.0893      35.0  5.267760   0.983607      1056.0  2.885246
2136   3.7578      24.0  5.061538   0.957692       781.0  3.003846
17546  2.4306      39.0  4.899209   1.069170      1990.0  3.932806
10051  3.2813      10.0  6.030928   1.159794       537.0  2.768041
3627   4.0950      36.0  5.407166   0.980456      1225.0  3.990228
[2.157 0.692 2.891 ... 1.048 1.407 1.26 ]


In [40]:
# [Answer to Ex. 12.2.2]
test_mse = []
train_mse = []
parameters = []
lambda_ = np.logspace(-4,4, 9)

lasso = Lasso()
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)
print("training score:", train_score)
print("test score: ", test_score)
print("number of features used: ", coeff_used)

lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,y_train)
train_score001=lasso001.score(X_train,y_train)
test_score001=lasso001.score(X_test,y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)

#print "training score for alpha=0.01:", train_score001 
#print "test score for alpha =0.01: ", test_score001
#print "number of features used: for alpha =0.01:", coeff_used001

#lasso00001 = Lasso(alpha=0.0001, max_iter=10e5)
#lasso00001.fit(X_train,y_train)train_score00001=lasso00001.score(X_train,y_train)
#test_score00001=lasso00001.score(X_test,y_test)
#coeff_used00001 = np.sum(lasso00001.coef_!=0)print "training score for alpha=0.0001:", train_score00001 
#print "test score for alpha =0.0001: ", test_score00001
#print "number of features used: for alpha =0.0001:", coeff_used00001lr = LinearRegression()
#lr.fit(X_train,y_train)
#lr_train_score=lr.score(X_train,y_train)
#lr_test_score=lr.score(X_test,y_test)print "LR training score:", lr_train_score 
#print "LR test score: ", lr_test_scoreplt.subplot(1,2,1)
#plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
#plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
#
#plt.xlabel('Coefficient Index',fontsize=16)
#plt.ylabel('Coefficient Magnitude',fontsize=16)
#plt.legend(fontsize=13,loc=4)plt.subplot(1,2,2)plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
#plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
#plt.plot(lasso00001.coef_,alpha=0.8,linestyle='none',marker='v',markersize=6,color='black',label=r'Lasso; $\alpha = 0.00001$') # alpha here is for transparency
#plt.plot(lr.coef_,alpha=0.7,linestyle='none',marker='o',markersize=5,color='green',label='Linear Regression',zorder=2)plt.xlabel('Coefficient Index',fontsize=16)
#plt.ylabel('Coefficient Magnitude',fontsize=16)
#plt.legend(fontsize=13,loc=4)
#plt.tight_layout()
#plt.show()

training score: 0.27839061941137133
test score:  0.28268553951454256
number of features used:  3


> **Ex.12.2.3**: Make a plot with on the x-axis and the RMSE measures on the y-axis. What happens to RMSE for train and test data as $\lambda$ increases? The x-axis should be log scaled. Which one are we interested in minimizing? 

> Bonus: Can you find the lambda that gives the lowest MSE-test score?

In [14]:
# [Answer to Ex. 12.2.3]